In [1]:
# !pip install lightgbm
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

import bytedtqs
from pytqs import tqs

mpl.rcParams['figure.dpi'] = 300
sns.set_style("whitegrid")
pd.options.display.max_columns = None
%config Completer.use_jedi = False

## Data Cleaning

In [2]:
appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
user_id = 'wuweiwei.1012'

sql_txt = """ 

set tqs.query.result.max.lines = 1000000000;

select * from dm_eo_test.recite_word_new_user limit 1000000000
"""

client = tqs.TQSClient(appId=appId, appKey=appKey)
job = client.execute_query(username = user_id , sql = sql_txt)
data = pd.read_csv(job['resultUrl'])

df_raw = data

[2021-04-15 20:22:28,686] - [INFO] - tqs_task_id: 248000103, engine: Hive, status: Created, progress : Submitting
[2021-04-15 20:22:30,712] - [INFO] - tqs_task_id: 248000103, execute id: tqs_248000103-1618489349220
[2021-04-15 20:22:30,713] - [INFO] - application tracking url: http://presto-history.byted.org/proxy/20210415_122229_96581_default
[2021-04-15 20:22:30,713] - [INFO] - tqs_task_id: 248000103, engine: Presto, status: Processing, progress : Accepted
[2021-04-15 20:22:32,740] - [INFO] - tqs_task_id: 248000103, engine: Presto, status: Processing, progress : Accepted
[2021-04-15 20:22:34,831] - [INFO] - tqs_task_id: 248000103, engine: Presto, status: Processing, progress : Accepted
[2021-04-15 20:22:36,858] - [INFO] - tqs_task_id: 248000103, engine: Presto, status: Processing, progress : Accepted
[2021-04-15 20:22:38,882] - [INFO] - tqs_task_id: 248000103, engine: Presto, status: Processing, progress : Accepted
[2021-04-15 20:22:40,905] - [INFO] - tqs_task_id: 248000103, engine: 

In [3]:
df_raw.head()

,user_id,install_date,new_dt,gap_days,enter_group,occupation,first_milestone_name,city_level,age,edu,gender,channel_group,study_mins,aim_module_user,study_dur_last,study_dur_last2,study_dur_last3,study_dur_last4,study_dur_last5,study_dur_next
0,435115798798,2021-03-20,2021-03-20,0,0,白领上班族,A2中,新一线,31-40,NaN,male,Google-FB,0,0,NaN,NaN,NaN,NaN,NaN,0.0
1,435115798798,2021-03-20,2021-03-23,3,0,白领上班族,A2中,新一线,31-40,high,male,Google-FB,0,0,0.0,0.0,0.0,NaN,NaN,0.0
2,435115798798,2021-03-20,2021-03-24,4,0,白领上班族,A2中,新一线,31-40,NaN,male,Google-FB,0,0,0.0,0.0,0.0,0.0,NaN,0.0
3,435115798798,2021-03-20,2021-03-25,5,0,白领上班族,A2中,新一线,31-40,NaN,male,Google-FB,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,435115798798,2021-03-20,2021-03-26,6,0,白领上班族,A2中,新一线,31-40,high,male,Google-FB,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# fill na
category_cols = []

for c in df_raw.columns:
    if df_raw[c].dtype=='O':
        df_raw[c] = df_raw[c].fillna('unknown')
        category_cols.append(c)
    else:
        df_raw[c] = df_raw[c].fillna(0)

In [8]:
df_raw.groupby(['aim_module_user']).size()

aim_module_user
0    2712409
1     101762
dtype: int64

In [13]:
# add cum status
df_sorted = df_raw\
    .sort_values(by=['user_id', 'gap_days'])
df_sorted['cum_recite'] = np.where(df_sorted\
    .groupby(['user_id'])['aim_module_user'].cumsum() > 0, 1, 0)
# df_sorted = df_raw

In [33]:
for _gap_days in range(0,5):
    df_train = df_sorted[lambda x: (x['gap_days']==_gap_days) & (x['enter_group']==0)].reset_index()
    
    train_x = df_train[_x]
    train_y = np.where(df_train[_y] > 0, 1, 0)
    
    # --------------- SEP ---------------
    # label encoding
    label_encoders = {}
    for c in categorial_x:
        encoder = LabelEncoder()
        train_x[c] = encoder.fit_transform(df_train[c].values)
        label_encoders[c] = encoder

    # --------------- SEP ---------------
    # counter-factual data
    train_x_cf_treated = train_x.assign(cum_recite=lambda x: 1)
    train_x_cf_untreated = train_x.assign(cum_recite=lambda x: 0)
    
    # training classifier
    mod = LGBMClassifier()
    mod.fit(train_x, train_y)
    
    y_pred_train = mod.predict(train_x)

    # predict counter-factual results
    pred_cf_treated = mod.predict_proba(train_x_cf_treated)[:,1]
    pred_cf_untreated = mod.predict_proba(train_x_cf_untreated)[:,1]
    
    # counter-factual mean diff
    mean_t = pred_cf_treated.mean()
    mean_u = pred_cf_untreated.mean()
    
    # actual mean diff
    df_stats = df_train.groupby(['cum_recite']).apply(lambda x: pd.Series({
        'user_cnt': x.shape[0],
        'retention_rate': (x[_y]>0).mean()
    })).reset_index()
    
    macth_summary = macth_summary.append({"gap_days": _gap_days , \
                                          "auc": "{0:0.4f}". format(accuracy_score(train_y, y_pred_train)), \
                                          "test": "{:.2%}".format(mean_t), \
                                          "control": "{:.2%}".format(mean_u), \
                                          "uplift": "{:.2%}".format(mean_t - mean_u),\
                                          "test_st": "{:.2%}".format(df_stats.iloc[1,2]),\
                                          "control_st": "{:.2%}".format(df_stats.iloc[0,2])\
                                         }, ignore_index = True)


macth_summary

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,gap_days,auc,test,control,uplift,test_st,control_st
0,0,1.0000,0.00%,0.00%,0.00%,8.77%,27.27%
1,1,1.0000,0.00%,0.00%,0.00%,5.70%,20.31%
2,2,0.9999,0.00%,0.01%,-0.01%,4.10%,16.01%
3,3,1.0000,0.00%,0.00%,-0.00%,3.01%,12.41%
4,4,1.0000,0.00%,0.00%,-0.00%,1.69%,7.29%
5,0,0.9102,16.11%,10.50%,5.61%,8.77%,27.27%
6,1,0.9483,9.58%,7.68%,1.90%,5.70%,20.31%
7,2,0.9609,6.90%,5.85%,1.05%,4.10%,16.01%
8,3,0.9685,5.04%,4.49%,0.55%,3.01%,12.41%
9,4,0.9806,2.99%,2.56%,0.43%,1.69%,7.29%
